In [28]:
# Install kaggle API
!pip install kaggle

# Upload kaggle.json (API token)
from google.colab import files
files.upload()   # apni kaggle.json file yahan select karo

# Save kaggle.json in correct folder
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json


In [2]:
!kaggle datasets download -d chetankv/dogs-cats-images

Dataset URL: https://www.kaggle.com/datasets/chetankv/dogs-cats-images
License(s): CC0-1.0
 93% 403M/435M [00:00<00:00, 459MB/s]
100% 435M/435M [00:00<00:00, 504MB/s]


In [3]:
!unzip -q dogs-cats-images.zip -d dogs_cats_dataset


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import os


In [5]:
import os

# Training aur test directories ka path
train_dir = '/content/dogs_cats_dataset/dataset/training_set'
test_dir = '/content/dogs_cats_dataset/dataset/test_set'

def count_images(directory):
    total = 0
    for root, dirs, files in os.walk(directory):
        total += len([f for f in files if f.endswith(('.jpg', '.png', '.jpeg'))])
    return total

train_count = count_images(train_dir)
test_count = count_images(test_dir)
total_count = train_count + test_count

print(f"Training images: {train_count}")
print(f"Test/Validation images: {test_count}")
print(f"Total images in dataset: {total_count}")


Training images: 8000
Test/Validation images: 2000
Total images in dataset: 10000


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# No augmentation for test/validation
test_datagen = ImageDataGenerator(rescale=1./255)

# Generators
train_generator = train_datagen.flow_from_directory(
    '/content/dogs_cats_dataset/dataset/training_set',   # updated path
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/dogs_cats_dataset/dataset/test_set',    # updated path
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.35),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [10]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 51s 179ms/step - accuracy: 0.4860 - loss: 0.7062 - val_accuracy: 0.5480 - val_loss: 0.6825
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 167ms/step - accuracy: 0.5790 - loss: 0.6784 - val_accuracy: 0.5630 - val_loss: 0.7016
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 43s 174ms/step - accuracy: 0.6345 - loss: 0.6375 - val_accuracy: 0.6615 - val_loss: 0.6265
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 166ms/step - accuracy: 0.6663 - loss: 0.6192 - val_accuracy: 0.7060 - val_loss: 0.5733
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 168ms/step - accuracy: 0.6928 - loss: 0.5879 - val_accuracy: 0.7505 - val_loss: 0.5175
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 43s 170ms/step - accuracy: 0.6980 - loss: 0.5686 - val_accuracy: 0.7470 - val_loss: 0.5184
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 45s 181ms/step - accuracy: 0.7189 - loss: 0.5526 - val_accuracy: 0.7445 - val_loss: 0.5118
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 42s 168ms/step - accuracy: 0.7344 -

In [11]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.8878 - loss: 0.2846
Validation Loss: 0.26937034726142883
Validation Accuracy: 0.8939999938011169


In [32]:
# Single Cell: Cat or Dog Prediction
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

# Upload image
uploaded = files.upload()

for fn in uploaded.keys():
    img_path = fn
    print(f"Uploaded image: {img_path}")

    # Preprocess image
    img = image.load_img(img_path, target_size=(128, 128))  # same size as training
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # rescale
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension

    # Prediction
    prediction = model.predict(img_array)[0][0]  # output 0 to 1
    if prediction < 0.5:
        print("This is a CAT 🐱")
    else:
        print("This is a DOG 🐶")


Saving tapetum-lucidum-light-retina-night-vision-vertebrates.webp to tapetum-lucidum-light-retina-night-vision-vertebrates.webp
Uploaded image: tapetum-lucidum-light-retina-night-vision-vertebrates.webp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
This is a CAT 🐱
